### Loading Required Libraries

In [ ]:
from modules import PreProcessing, functions
from sqlite3 import connect 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain
from collections import Counter
import nltk
import warnings
warnings.filterwarnings("ignore")


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from spacy.lang.en.stop_words import STOP_WORDS as stopwords

# to ensure all changes are reflected 
# in-case jupyter notebook was already oppened

from importlib import reload
reload(PreProcessing)
reload(functions)

In [ ]:
import spacy.cli
# spacy.cli.download("en_core_web_md")

import spacy_fastlang

import spacy
from bertopic import BERTopic

### Section 1 - Create a Suport vector machine 

In [ ]:
databse_file = 'database.db'
main = connect(databse_file)

sql_statement_laod_data = '''
SELECT 
    reviews.id, 
    reviews.course_name,
    reviews.text

FROM reviews
'''

df = pd.read_sql_query(sql_statement_laod_data, main)

df.sample(5)

In [ ]:
df.isnull().sum() * 100 / len(df)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum() * 100 / len(df)

In [ ]:
# dfX = df.copy()

In [ ]:
# OOOPSIES
# df = dfX.copy()

In [ ]:
import re
def text_decoder(string):
    '''
        The following two lines of code are from Wim Feijen on StackOverFlow

        Wim Feijen
        March 26, 2021
        StackOverFlow: Python encoding/decoding problems
        https://stackoverflow.com/questions/27996448/python-encoding-decoding-problems
    '''
    bytes_string = bytes(string, encoding="raw_unicode_escape")
    text = bytes_string.decode("utf-8", "strict")
    return text

def lang_detect(clean_string):
    clean_string = nlp(clean_string)
    return clean_string._.language

def tokens_2_keep(text_column):
    tokens = text_column.split()
    return [token for token in tokens if re.match('^[a-z0-9]+$', token)]

nlp = spacy.load('en_core_web_md', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])
nlp.add_pipe("language_detector")

In [ ]:
%%time
df['language'] = df['text'].apply(lang_detect)
df = df[df['language'] == 'en']
df['cleaned_text'] = df['text'].apply(text_decoder)
# df.sample(5)

In [ ]:
df[df['id'] == 19301]

In [ ]:
df[df['id'] == 29208]

In [ ]:
def token_counter(text):
    return len(text.split())

df['token_count'] = df['cleaned_text'].apply(token_counter)
# df[df['token_count'] == 1].sample(5)

In [ ]:
# calculate how many single word documents exists in corpus
round(df[df['token_count'] == 1].shape[0] / df.shape[0], 4)

It looks like we are fine from overfitting to the single word documents/reviews, but to stay safe we will create two different models.  One with single word docs and one without single word docs.

In [ ]:
text_ =  df.loc[df['id'] == 38359, 'cleaned_text'].to_string(index=False)
text_

In [ ]:
view = df.loc[df['id'] == 21,'cleaned_text'].to_string(index=False)
view

In [ ]:
df['cleaned_text'] = df['cleaned_text'].str.replace('\\n','')
text_ =  df.loc[df['id'] == 38359, 'cleaned_text'].to_string(index=False)
text_

In [ ]:
df = df[['course_name', 'cleaned_text', 'token_count']]
df.sample(10)

# Model

In [ ]:
nlp = spacy.load('en_core_web_md', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

In [ ]:
# topic_model = BERTopic(embedding_model=nlp, nr_topics='auto')
# topics, probs = topic_model.fit_transform(df['cleaned_text'])
#
# fig = topic_model.visualize_topics()
# fig.show()

In [ ]:
docs = df.loc[df['token_count'] > 4, 'cleaned_text']
topic_model = BERTopic(embedding_model=nlp, nr_topics='auto')
topics, probs = topic_model.fit_transform(docs)

fig = topic_model.visualize_topics()
fig.show()

In [ ]:
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=(2,3))
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

fig = topic_model.visualize_topics()
fig.show()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
# # Further reduce topics
# topic_model.reduce_topics(docs, nr_topics=50)
fig = topic_model.visualize_topics()
fig.show()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topic_model.get_topic(10)

In [ ]:
break

In [ ]:
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=(1, 5))
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

fig = topic_model.visualize_topics()
fig.show()

In [ ]:
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=(2, 5))
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

fig = topic_model.visualize_topics()
fig.show()

In [ ]:
# Further reduce topics
topic_model.reduce_topics(docs, nr_topics=50)

fig = topic_model.visualize_topics()
fig.show()

In [ ]:
# Further reduce topics
topic_model.reduce_topics(docs, nr_topics=30)

fig = topic_model.visualize_topics()
fig.show()